# **Subset geospatial data and calculate landscape metrics**

By Bridget Bittmann

Date created: March 14, 2022

Date editted: March 28, 2022

In [ ]:
## IMPORT PACKAGES ##

import numpy as np #basic computation
!pip install geopandas
import geopandas as gpd #geopandas for .shp
import matplotlib.pyplot as plt #to create plots
import pandas as pd #to create dataframes and export .csv
!pip install rasterio
import rasterio as rso #import GeoTiff files
from rasterio.mask import mask #to crop data to a boundary
from rasterio.plot import show #to plot the image
from rasterio.crs import CRS
from shapely.ops import unary_union #creates boundary of shapefile
import json #imports metadata
!pip install rioxarray #to clip rasters to a .shp file
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
!pip install pylandstats
import pylandstats #to perform landscape metrics
from pylandstats import landscape
from pylandstats import SpatioTemporalAnalysis #to calculate landscape metrics through time
import glob
import os

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
## NAVIGATE TO YOUR DIRECTORY ##
%cd gdrive/MyDrive/spatial_colab/datasets/
%ls

/content/gdrive/MyDrive/spatial_colab/datasets
2005a.tif  2010.tif  2016.tif    2021.tif               irrig_lbrb/
2006.tif   2011.tif  2017.tif    Boise_CDL/             IrrMapper/
2007a.tif  2012.tif  2018.tif    CDL_reproject/         LBRB_shp/
2007b.tif  2013.tif  2019.tif    CDL_subsets/           lcmap_files/
2008.tif   2014.tif  2020.tif    diversion_timeseries/  masked/
2009.tif   2015.tif  2021_clip/  irrigation_companies/  subset_test_shp/


In [ ]:
## ---------------------- ##
## SUBSET GEOSPATIAL DATA ##
## ---------------------- ##

shp_file = gpd.read_file('subset_test_shp/subset.shp') #open shapefile
names = shp_file['DIV_NAME_']
files = glob.glob('lcmap_files/*.tiff') #get all the years of cdl imagery
data =[]
for i in range(len(files)):
  data.append(rso.open(files[i])) #open cdl image and append to a list
print(data)
shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
years = np.arange(1985, 2021) #years of LCMAP data
collection = []
for i in range(len(shp)):
  for n in range(len(years)):
    dataset = data[n]
    year_out = dataset.name[21:-15]
    extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
    coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
    out_img, out_transform = mask(dataset=data[n], shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
    out_meta = data[n].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform})
    # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
    in_file = files[n]
    out_filename = os.path.join('lcmap_files/lcmap_masked/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
    # Save clipped land cover coverage THIS WILL OVERWRITE FILES
    out_tiff = rso.open(out_filename, 'w', **out_meta)
    out_tiff.write(np.squeeze(out_img, 0), 1)
    out_tiff.close()
    collection.append(out_img)

[<open DatasetReader name='lcmap_files/LCMAP_CU_1987_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1986_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1989_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1988_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1990_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1992_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1991_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1993_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1994_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1995_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1996_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1997_V12_LCPRI.tiff' mode='r'>, <open DatasetReader name='lcmap_files/LCMAP_CU_1999

In [95]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##

temporal_group = []
for i in range(len(names)):
  files = glob.glob('lcmap_files/lcmap_masked/'+names[i]+'_*.tif') #name for all the csv files
  sta = SpatioTemporalAnalysis(files, dates=years, nodata=0) #import all CDL rasters and mask
  temporal_group.append(sta)

In [120]:
# ------------------------------- #
# CALCULATE THE CLASS PROPORTIONS #
# ------------------------------- #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('lcmap_files/proportions/'+names[i]+'.csv')
  proportions.append(df)

In [ ]:
# ------------------------------- #
# CALCULATE CONFIGURATION METRICS #
# ------------------------------- #

config = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['contagion', 'largest_patch_index', 'shannon_diversity_index'])
  config.append(df)

In [91]:
display(config[2])

metric,contagion,largest_patch_index,shannon_diversity_index
dates,,,
1985,86.247131,72.430748,0.342933
1986,86.211281,72.480867,0.343864
1987,86.109194,72.381984,0.346545
1988,86.216766,72.407721,0.343854
1989,84.912468,72.358957,0.347078
1990,86.008463,72.354893,0.349918
1991,86.070280,72.354893,0.348069
1992,86.066360,72.400948,0.349662
1993,86.038689,72.432103,0.351848


In [101]:
display(proportions[0])

metric           proportion_of_landscape
class_val dates                         
1         1985                  9.039825
          1986                  8.835487
          1987                  9.369745
          1988                  9.193078
          1989                  9.548541
...                                  ...
8         2016                  0.000000
          2017                  0.000000
          2018                  0.000000
          2019                  0.000000
          2020                  0.000000

[252 rows x 1 columns]

In [150]:
# ----------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics #
# ----------------------------------------------------------------- #

# Import csv files into a list of dataframes

files = glob.glob('lcmap_files/proportions/*.csv')
proportions = []
for i in files:
  data = pd.read_csv(i)
  proportions.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2]
  df['class3_grass-shrub'] = prop['proportion_of_landscape'][prop['class_val'] == 3]
  df['class4_treecover']= prop['proportion_of_landscape'][prop['class_val'] == 4]
  df['class5_water'] = prop['proportion_of_landscape'][prop['class_val'] == 5]
  df['class6_wetland'] = prop['proportion_of_landscape'][prop['class_val'] == 6]
  df['class7_ice-snow'] = prop['proportion_of_landscape'][prop['class_val'] == 7]
  df['class8_barren'] = prop['proportion_of_landscape'][prop['class_val'] == 8]
  new_df.append(df)




In [ ]:
## MERGE TWO DATAFRAMES ##